In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Solving environment: done

# All requested packages already installed.



/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jupyterlab/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Solving environment: \ 

In [ ]:
import wikipedia as wp

In [ ]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)

In [6]:
# Create a new variable called 'header' from the first row of the dataset
header = df.iloc[0]
# Replace the dataframe with a new one which does not contain the first row
df = df[1:]
# rinizializzo l'indice a 0
df.index -= 1
# Rename the dataframe's column values with the header variable
df=df.rename(columns = header)

In [7]:
# Drop a row by condition
df=df[df.Borough != 'Not assigned']
# rinizializzo l'indice a 0 dopo il drop
df = df.reset_index(drop=True)

In [8]:
for i in range(len(df)):
    if df.Neighbourhood[i] == 'Not assigned':
        df.Neighbourhood[i] = df.Borough[i]

In [9]:
df_grouped= df.groupby(['Postcode', 'Borough'], as_index=False)['Neighbourhood'].agg({'Neighbourhood': lambda x: ','.join(x)})

In [10]:
df_grouped.shape

(103, 3)

In [11]:
# import geospatial coordinates from related csv
coord = pd.read_csv('Geospatial_Coordinates.csv')

FileNotFoundError: File b'Geospatial_Coordinates.csv' does not exist

In [ ]:
#left join of the two dataframe
df_merge = pd.merge(df_grouped,coord,left_on='Postcode',right_on='Postal Code',how='left')
#drop of the duplicated column
df_merge = df_merge.drop(['Postal Code'],axis=1)

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [ ]:
toronto_data = df_merge[df_merge['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

In [ ]:
# create map of Manhattan using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto2)  
    
map_toronto2

Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'GS1WCZO4VQ1JZ5YWU3SBQCWCG3JUK5U1CUBKQZS3VIIKLAZW' # your Foursquare ID
CLIENT_SECRET = 'JHR1DXNXIFBMTLIV52V25OCGRVGFZM3RMTRX0WPDQ12DRK1B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
toronto_data.loc[0, 'Neighbourhood']